In [ ]:
import numpy as np
from numba import njit


## inside this function, every term has been made into a (N.M)-numpy array for matrix calculations



def seiv(y,t,
          r,
         beta,
         phi,
         tau,
         NE):
    
    
    # make these arrays (N,) --> (N,1) arrays, ie, vectors
    y = y.reshape(-1,1)
    
    # unpack variables 
    # these variables vary over time
    
    S = y[0:NH] # 1d vector
    Emat = y[NH : NH + NH*NV*NE]  #should be a 3d tensor
    Emat = Emat.reshape((NE,NH,NV)) # reshaped into a 3d tensor
    # Emat[i,:,:] is the i'th box -- so first element for the box number
    Imat = y[NH + NH*NV*NE : NH + NH*NV*NE + NH*NV] #2d matrix
    Imat = Imat.reshape((NH,NV))
    V = y[NH + NH*NV*NE + NH*NV : NH + NH*NV*NE + NH*NV + NV ]  #1d matrix
    D = y[NH + NH*NV*NE + NH*NV + NV] # a number
    
    

    
    
    # pack parameters into matrices.
    r = np.array([[r1,r2,r3,r4,r5]]).T #transposed it, so this is a column vector now.
    
    beta = np.array([[0, beta12, 0, 0, 0 ],
                     [beta21, beta22, beta23, 0, 0],
                     [0, 0, beta33, 0 , 0],
                     [0, 0, 0, beta44, beta45],
                     [0, 0, 0, beta54, beta55]])
    
    tau = np.array([[0, tau12, 0, 0, 0 ],
                     [tau21, tau22, tau23, 0, 0],
                     [0, 0, tau33, 0 , 0],
                     [0, 0, 0, tau44, tau45],
                     [0, 0, 0, tau54, tau55]])
    
    phi = np.array([[0, phi12, 0, 0, 0 ],
                     [phi21, phi22, phi23, 0, 0],
                     [0, 0, phi33, 0 , 0],
                     [0, 0, 0, phi44, phi45],
                     [0, 0, 0, phi54, phi55]])
    

    
    Dc = np.array([[Dc1,Dc2,Dc3,Dc4,Dc5]]).T
    #etaeff = (NE+1) * np.where(tau != 0, 1 / tau, 0)  # tested: works fine
    etaeff = (NE+1)*np.array([[0, 1/tau12, 0, 0, 0 ],
                     [1/tau21, 1/tau22, 1/tau23, 0, 0],
                     [0, 0, 1/tau33, 0 , 0],
                     [0, 0, 0, 1/tau44, 1/tau45],
                     [0, 0, 0, 1/tau54, 1/tau55]])
    

    
    #set up variables for the differential equation
    Sdeb = S/(1+(D/Dc)**2) #element wise division
    N = S + np.sum(Emat,axis=0)@np.ones((NV,1)) + Imat@np.ones((NV,1))  #sum of all the different types of host cells (is a column vector of same size as S)
    # we summed the 0 axis, for boxes, then we multiplied with a col vector of size NV, means summed at each row level
    # means at each row, summed elements of each columns, so viruses summer up, and we net N_i for each host
    Ndeb = N/(1+(D/Dc)**2) #number of host of each species scaled by debris factor.
    
    
    
    ## the differential eqauations
    dS = r*S - Sdeb * (phi@V) #host change
    
    dEmat = np.zeros((NE,NH,NV)) # initializing as zero tensor, otherwise will give undefined error
    #dEmat[0,:,:] = Sdeb * ((phi*beta)@V) - etaeff*Emat[0,:,:]
    #dEmat[0,:,:] =  phi*(Sdeb@V.T) - etaeff*Emat[0,:,:]
    dEmat[0,:,:] = (phi*(Sdeb@V.T) - etaeff*Emat[0,:,:]).reshape((NH, NV))

    
    #vectorize this
    dEmat[1:,:,:] = etaeff * (-Emat[1:,:,:] + Emat[0:-1,:,:])  # note the signs 
    dImat = etaeff * (Emat[NE-1,:,:] - Imat) #element wise multiplication
    
    # for the viruses we need to sum up all the row elements for each column 
    # sum_{j=1}^{NH} Imat'_{kj} = Imat_k 
    #for the part of viral ads, transpose phi and multiply, as sum over the hosts
    dV = (beta * etaeff * Imat).T @ np.ones((NH,1)) - V * (phi.T @ Ndeb)
    dD = np.sum(etaeff* Imat)
    
    
    dydt = np.concatenate((dS.flatten(), dEmat.flatten(), dImat.flatten(), dV.flatten(), [dD]))
    return dydt